In [1]:
import networkx as nx
import numpy as np
from copy import deepcopy

##########################################

def inverse_hcat(graph,degrees):
  #degrees is a dictionary with keys as argument indices and values as degrees
  d=np.zeros(len(degrees))
  for deg in degrees:
    d[deg]=degrees[deg]
  adj=nx.convert_matrix.to_numpy_array(graph)
  adj=adj.T
  M=np.diag(d)
  return d+np.matmul(M,np.matmul(adj,d))

def inverse_card(graph,degrees):
  d=np.zeros(len(degrees))
  for deg in degrees:
    d[deg]=degrees[deg]

  adj=nx.convert_matrix.to_numpy_array(graph)
  adj=adj.T
  att=np.sum(adj,axis=1)
  att_m=np.diag(np.sum(adj,axis=1))
  div=np.zeros(len(d))
  np.divide(np.matmul(np.diag(d),np.matmul(adj,d)),att,where=att!=0,out=div)
  lhs=d+np.matmul(att_m,d)+div
  return lhs

def inverse_mbs(graph,degrees):
  adj=nx.convert_matrix.to_numpy_array(graph)
  adj=adj.T

  d=np.zeros(len(degrees))
  for deg in degrees:
    d[deg]=degrees[deg]
  M=np.diag(d)

  return d+np.matmul(M,np.max(np.matmul(adj,M),axis=1))

##########################################
class CAF:
    def __init__(self,af,intervals):
        self.af = af #this is a networkx graph
        self.intervals = intervals #this is a dictionary of min/max tuples with arguments as keys
    
    def is_rational(self,semantics):
        #semantics is one of hcat, card, mbs
        #a CAF is irrational if there is no weighting s.t. sigma(a_i) \in I(a_i) for all a_i
        #so we need to do the inverse semantics thing and check whether all arguments are in the interval.
        #we can do this by checking whether the minimum of the CAF is valid (assuming axial-radiality)
        #deg = [0] * len(self.af.nodes)
        deg = {}
        for i in range(len(self.af.nodes)):
           deg[i]=min(self.intervals[i])
        weights = semantics(self.af,deg)
        if max(weights)>1 or min(weights)<0:
            return False
        else:
            return True
    
    def is_fully_rational(self,semantics):
       #as above, but check max interval
        deg = [0] * len(self.af.nodes)
        for i in range(len(self.af.nodes)):
           deg[i]=max(self.intervals[i])
        weights = semantics(self.af,deg)
        if max(weights)>1 or min(weights)<0:
            return False
        else:
            return True
      
    def is_epsilon_rational(self,semantics,epsilon):
      for i in range(len(self.af.nodes)):
          if not epsilon <= max(self.intervals[i])-min(self.intervals[i]):
             return False
          new_caf = deepcopy(self)
          new_caf.intervals[i]=(self.intervals[i][1]-epsilon,self.intervals[i][1])
          if not new_caf.is_rational(semantics):
              return False
      return True
    
    def best_refinement(self,semantics,epsilon):
        for a in self.arguments:
            new_caf=deepcopy(self)
            new_caf.intervals[a]=(self.intervals[a][1],self.intervals[a][1])
            if not new_caf.is_rational(semantics):
                l = min(self.intervals[a])
                r = max(self.intervals[a])
                found = False
                while not found:
                   m = (l+r)/2
                   new_caf2 = deepcopy(new_caf)
                   new_caf2.intervals[a]=(m,m)
                   if not new_caf2.is_rational(semantics):
                      r = m
                      if (l-2/2)<epsilon:
                         self.intervals[a]=(min(self.intervals[a]),m)
                         found = True
                   else:
                      l = m

In [2]:
NUM_TRIALS = 100

def make_random_graph(size):
    g = nx.fast_gnp_random_graph(size,0.5)
    intervals = {}
    for i in range(size):
        intervals[i]=(0,1)
    return CAF(g,intervals)

for i in range(NUM_TRIALS):
    #generate a random CAF
    caf = make_random_graph(10)
    while not caf.is_rational(inverse_hcat) and caf.is_epsilon_rational(inverse_hcat,0.01):
        caf = make_random_graph(10)
        caf.best_refinement(inverse_hcat,0.01)
        if not caf.is_epsilon_rational(inverse_hcat,0.01):
            print("Error")
            break




The next part computes a line starting at the "bottom left" corner of the interval and with gradient equal to 1/cost per argument. It then does a bisection search along this line for a point in the shape, and returns the new interval.

In [36]:

def line_end(caf):
    #returns the corner of the CAF closest to the origin as a vector
    o=[]
    for a in caf.af.nodes:
        o.append(caf.intervals[a][0])
    return np.array(o)

def line_gradient(arg_costs):
    o=[]
    for i in range(len(arg_costs)):
        o.append(1/arg_costs[i])
    return np.array(o)

def compute_start_l(caf,arg_costs):
    #finds the point where all arguments are <=0 when moving along line_gradient from line_end. Do this by working out t for each argument.
    #Compute $t as per above
    min=1
    for a in caf.af.nodes:
        t = 1-arg_costs[a]*caf.intervals[a][0]
        if t<min:
            min=t
    return min

#now we can do a bisection search between min and 1 to find the point. For each argument, if t makes it's value below 0, we set it to 0.
def make_rational(caf,arg_costs,epsilon):
    l=compute_start_l(caf,arg_costs)
    lg = line_gradient(arg_costs)
    h=1
    
    new_caf = deepcopy(caf)
    while (h-l)>epsilon:

        m=(l+h)/2
        #print(l,h,m,new_caf.intervals)
        for a in range(len(new_caf.af.nodes)):
            new_caf.intervals[a]=(max(0,
                                     m*lg[a] + caf.intervals[a][0]),
                                     caf.intervals[a][1])
            
            
        if not new_caf.is_rational(inverse_hcat):
            h=m
        else:
            l=m
        
    for a in range(len(new_caf.af.nodes)):
        new_caf.intervals[a]=max(0,l*lg[a]+caf.intervals[a][0]),caf.intervals[a][1]
    return new_caf

def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]

def make_rational_heuristic_strategy2(caf,arg_costs,epsilon):
    #here we move only a subset of arguments
    best=999999999
    best_set=[]
    best_caf=None
    for p in powerset(caf.af.nodes):
        l = compute_start_l(caf,arg_costs)
        lg = line_gradient(arg_costs)
        for a in caf.af.nodes:
            if a not in p:
                lg[a]=0
        
        h=1
        new_caf = deepcopy(caf)
        while (h-l)>epsilon:
            m=(l+h)/2
            for a in range(len(new_caf.af.nodes)):
                new_caf.intervals[a]=(max(0,
                                         m*lg[a] + caf.intervals[a][0]),
                                         caf.intervals[a][1])
            if not new_caf.is_rational(inverse_hcat):
                h=m
            else:
                l=m
        
        for a in range(len(new_caf.af.nodes)):
            new_caf.intervals[a]=max(0,l*lg[a]+caf.intervals[a][0]),caf.intervals[a][1]
        
        if new_caf.is_rational(inverse_hcat):
            cost=0
            for a in p:
                cost+=(1-l)*arg_costs[a]
            if cost<best:
                best=cost
                best_set=p
                best_caf=new_caf

    return best,best_set,new_caf
        

In [37]:
#test the make_rational function
caf = make_random_graph(3)
caf.intervals={0:(1,1),1:(1,1),2:(1,1)}
arg_costs=[3,2,1]

print(make_rational_heuristic_strategy2(caf,arg_costs,0.01))


(6, [1], <__main__.CAF object at 0x1077f0fa0>)


Another strategy is to have a line start from the origin to the point, and do a search along that one.

In [17]:
#we can use the line_end function and then do a bisection search between 0 and 1.
def make_rational2(caf,epsilon):
    l=0
    h=1
    new_caf = deepcopy(caf)
    while (h-l)>epsilon:
        m=(l+h)/2
        print(l,h,m,new_caf.intervals)
        for a in range(len(new_caf.af.nodes)):
            new_caf.intervals[a]=(m*caf.intervals[a][0],caf.intervals[a][1])
        if not new_caf.is_rational(inverse_hcat):
            h=m
        else:
            l=m
            
    for a in range(len(new_caf.af.nodes)):
        new_caf.intervals[a]=l*caf.intervals[a][0],caf.intervals[a][1]
    return new_caf

In [24]:

caf = make_random_graph(3)
caf.intervals={0:(1,1),1:(1,1),2:(1,1)}
arg_costs=[3,2,1]

new_caf=make_rational2(caf,0.01)
print(new_caf.intervals)

0 1 0.5 {0: (1, 1), 1: (1, 1), 2: (1, 1)}
0.5 1 0.75 {0: (0.5, 1), 1: (0.5, 1), 2: (0.5, 1)}
0.5 0.75 0.625 {0: (0.75, 1), 1: (0.75, 1), 2: (0.75, 1)}
0.5 0.625 0.5625 {0: (0.625, 1), 1: (0.625, 1), 2: (0.625, 1)}
0.5 0.5625 0.53125 {0: (0.5625, 1), 1: (0.5625, 1), 2: (0.5625, 1)}
0.5 0.53125 0.515625 {0: (0.53125, 1), 1: (0.53125, 1), 2: (0.53125, 1)}
0.5 0.515625 0.5078125 {0: (0.515625, 1), 1: (0.515625, 1), 2: (0.515625, 1)}
{0: (0.5, 1), 1: (0.5, 1), 2: (0.5, 1)}


{0: (0.5, 1), 1: (0.5, 1), 2: (0.5, 1)}

In [28]:
list(powerset([1,2,3]))

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]